In [3]:
import urllib.request
import json
import pandas as pd
import numpy as np
# Make the request to the API
req = urllib.request.Request('https://livescore-api.com/api-client/matches/live.json?&key=h5v5oR8HzTPeQ41G&secret=3I0FK43hPoL81itjoZoMRxuqqjEgoGXr')
response = urllib.request.urlopen(req)
# Read the response and convert it from JSON to a Python dictionary
data = json.loads(response.read())
# Assuming the relevant data is in a key named 'data' (you will need to adjust this based on the actual structure of the JSON response)
# Convert the dictionary to a pandas DataFrame
dframe = pd.DataFrame(data['data']['match'])
dframe['home_id'] = dframe['home'].apply(lambda x: x['id'])
dframe['away_id'] = dframe['away'].apply(lambda x: x['id'])
dframe['home_name'] = dframe['home'].apply(lambda x: x['name'])
dframe['away_name'] = dframe['away'].apply(lambda x: x['name'])

dframe['score'] = dframe['scores'].apply(lambda x: x['score'])
columns_to_drop = ['federation','competition','scheduled','country','odds','added','last_changed','away','home','outcomes','scores','urls']
# Loại bỏ các cột không mong muốn
dframe = dframe.drop(columns=columns_to_drop)
pd.set_option('display.max_columns', None)
dframe['team1.overall_form'] = None
dframe['team1.h2h_form'] = None
dframe['team2.overall_form'] = None
dframe['team2.h2h_form'] = None

# Function to make the request to the API
def fetch_head2head_data(team1_id, team2_id):
    url = f'https://livescore-api.com/api-client/teams/head2head.json?team1_id={team1_id}&team2_id={team2_id}&key=h5v5oR8HzTPeQ41G&secret=3I0FK43hPoL81itjoZoMRxuqqjEgoGXr'
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = json.loads(response.read())
    df = pd.DataFrame(data)
    return df

# Function to extract and normalize JSON data
def extract_and_normalize(json_data):
    normalized_data = pd.json_normalize(json_data)
    return normalized_data

# Iterate over each row in the DataFrame
# Create an empty list to store the row data
row_data = []

for index, row in dframe.iterrows():
    team1_id = row['home_id']
    team2_id = row['away_id']
    
    # Fetch data from the API
    df = fetch_head2head_data(team1_id, team2_id)
    
    # Extract and normalize the data
    team1_overall_form = df['data'].iloc[0]
    team1_h2h_form = df['data'].iloc[0]
    team2_overall_form = df['data'].iloc[1]
    team2_h2h_form = df['data'].iloc[1]
    
    # Normalize the data
    team1_overall_form = pd.json_normalize(team1_overall_form)
    team1_h2h_form = pd.json_normalize(team1_h2h_form)
    team2_overall_form = pd.json_normalize(team2_overall_form)
    team2_h2h_form = pd.json_normalize(team2_h2h_form)
    
    # Update the original DataFrame
    dframe.at[index, 'team1.overall_form'] = team1_overall_form['overall_form'].iloc[0]
    dframe.at[index, 'team1.h2h_form'] = team1_h2h_form['h2h_form'].iloc[0]
    dframe.at[index, 'team2.overall_form'] = team2_overall_form['overall_form'].iloc[0]
    dframe.at[index, 'team2.h2h_form'] = team2_h2h_form['h2h_form'].iloc[0]

# Display the updated DataFrame
dframe


,id,fixture_id,status,time,location,home_id,away_id,home_name,away_name,score,team1.overall_form,team1.h2h_form,team2.overall_form,team2.h2h_form
0,532346,1694565,FINISHED,FT,Estádio Felipe Santiago,6896,3963,Floresta,Ferroviário CE,1 - 2,"[L, L, L, L, L, W]","[W, W, W]","[L, L, D, L, W, D]","[L, L, L]"
1,532347,1687492,IN PLAY,89,Estadio Hernando Siles,276,1553,Bolivar,Palestino,3 - 1,"[W, D, W, L, W, D]",[W],"[L, L, D, W, W, L]",[L]
2,532348,1687493,ADDED TIME,90+,Estadio Jornalista Mário Filho (Maracanã),264,755,Flamengo,Millonarios,3 - 0,"[W, W, W, L, D, W]",[D],"[D, D, L, D, W, D]",[D]
3,532349,1687855,ADDED TIME,90+,,5308,7991,Independiente Juniors,San Antonio,2 - 0,"[L, L, D, D, W, W]",[D],"[D, D, D, D, W, W]",[D]
4,532350,1688520,IN PLAY,67,Estadio Bicentenario Francisco Sánchez Rumoroso,4050,3239,Coquimbo Unido,Bragantino,1 - 0,"[W, W, L, D, W, W]",[L],"[W, W, L, W, D, D]",[W]
5,532351,1688521,IN PLAY,66,Estadio Presidente Juan Domingo Perón,935,1289,Racing Club,Luqueno,2 - 0,"[W, W, W, D, L, L]",[W],"[W, L, W, L, D, D]",[L]
6,532352,1695817,IN PLAY,63,Estadio Olímpico Jaime Morón León,4058,1578,Real Cartagena,Atletico Huila,0 - 0,"[W, D, L, L, L, W]","[W, L, L, W, D, L]","[L, W, W, L, W, L]","[L, W, W, L, D, W]"
7,532353,1688523,IN PLAY,65,Estadio Jocay,2397,6566,Delfin,Real Tomayapo,2 - 3,"[W, L, L, D, L, L]",[W],"[L, L, L, W, L, D]",[L]
8,532354,1688522,IN PLAY,66,Estádio José Pinheiro Borda,2416,451,Internacional,Belgrano Cordoba,1 - 2,"[D, W, L, W, W, D]",[D],"[W, L, W, D, D, W]",[D]


In [4]:
import urllib.request
import json
import pandas as pd
import numpy as np

df=dframe.copy()
def calculate_ratios(form):
    win_count = form.count('W')
    draw_count = form.count('D')
    loss_count = form.count('L')
    total = len(form)
    win_ratio = win_count / 6
    draw_ratio = draw_count / 6
    loss_ratio = loss_count / 6
    return win_ratio, draw_ratio, loss_ratio
df['team1.overall_win_ratio'], df['team1.overall_draw_ratio'], df['team1.overall_loss_ratio'] = zip(*df['team1.overall_form'].apply(calculate_ratios))
df['team1.h2h_win_ratio'], df['team1.h2h_draw_ratio'], df['team1.h2h_loss_ratio'] = zip(*df['team1.h2h_form'].apply(calculate_ratios))
df['team2.overall_win_ratio'], df['team2.overall_draw_ratio'], df['team2.overall_loss_ratio'] = zip(*df['team2.overall_form'].apply(calculate_ratios))
df['team2.h2h_win_ratio'], df['team2.h2h_draw_ratio'], df['team2.h2h_loss_ratio'] = zip(*df['team2.h2h_form'].apply(calculate_ratios))
columns_to_keep = ['home_id','away_id','team1.overall_win_ratio', 'team1.overall_draw_ratio', 'team1.overall_loss_ratio',
                   'team1.h2h_win_ratio', 'team1.h2h_draw_ratio', 'team1.h2h_loss_ratio',
                   'team2.overall_draw_ratio','team2.overall_loss_ratio','team2.h2h_win_ratio',
                   'team2.h2h_draw_ratio','team2.h2h_loss_ratio']
df = df[columns_to_keep]

In [5]:
# Assuming your DataFrame is named 'dfram' and has a column named 'match_id'
api_key = 'h5v5oR8HzTPeQ41G'
api_secret = '3I0FK43hPoL81itjoZoMRxuqqjEgoGXr'

# Create an empty list to store the DataFrames for each match
match_data_list = []

# Iterate over each row in the DataFrame
for index, row in dframe.iterrows():
    match_id = row['id']

    # Make the request to the API
    url = f'https://livescore-api.com/api-client/matches/stats.json?match_id={match_id}&key={api_key}&secret={api_secret}'
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    # Read the response and convert it from JSON to a Python dictionary
    data = json.loads(response.read())
    # Assuming the relevant data is in a key named 'data' (adjust based on the actual JSON structure)
    # Convert the dictionary to a pandas DataFrame
    # Modify the line to include an index
    match_df = pd.DataFrame(data['data'], index=[match_id])
    # Add the match_id as a column to the DataFrame
    match_df['match_id'] = match_id
    # Append the DataFrame to the list
    match_data_list.append(match_df)
# Concatenate all the DataFrames into a single DataFrame
final_df = pd.concat(match_data_list, ignore_index=True)

In [6]:
def split_and_add_columns(final_df, dframe):
    # Define the columns to be split
    columns_to_split = [
        'yellow_cards', 'red_cards', 'substitutions', 'possesion', 'free_kicks',
        'goal_kicks', 'throw_ins', 'offsides', 'corners', 'shots_on_target',
        'shots_off_target', 'attempts_on_goal', 'saves', 'fauls', 'treatments',
        'penalties', 'shots_blocked', 'dangerous_attacks', 'attacks'
    ]

    # Iterate through each row in final_df
    for _, row in final_df.iterrows():
        match_id = row['match_id']

        # Find the corresponding row in dframe
        dframe_row = dframe[dframe['id'] == match_id]

        if not dframe_row.empty:
            index = dframe_row.index[0]

            # Split the values and add them to dframe
            for col in columns_to_split:
                if pd.notna(row[col]):
                    home_value, away_value = row[col].split(':')
                    dframe.at[index, f'home_{col}'] = home_value
                    dframe.at[index, f'away_{col}'] = away_value
                else:
                    dframe.at[index, f'home_{col}'] = None
                    dframe.at[index, f'away_{col}'] = None

# Apply the function
split_and_add_columns(final_df, dframe)
merged_df = pd.merge(dframe, df, on=['home_id', 'away_id'], how='left')
merged_df=merged_df.drop(columns=['team1.overall_form',	'team1.h2h_form','team2.overall_form','team2.h2h_form'])
merged_df=merged_df.fillna(0)

/tmp/ipykernel_134202/960478638.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df=merged_df.fillna(0)


In [7]:
merged_df = pd.merge(dframe, df, on=['home_id', 'away_id'], how='left')
merged_df=merged_df.drop(columns=['team1.overall_form',	'team1.h2h_form','team2.overall_form','team2.h2h_form'])
merged_df=merged_df.fillna(0)

/tmp/ipykernel_134202/1790838502.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df=merged_df.fillna(0)


In [8]:
# Split the 'score' column into 'ht_score_home' and 'ht_score_team'
merged_df[['ht_score_home', 'ht_score_team']] = merged_df['score'].str.split('-', expand=True)

# Convert the new columns to integers
merged_df['ht_score_home'] = merged_df['ht_score_home'].astype(float)
merged_df['ht_score_team'] = merged_df['ht_score_team'].astype(float)

# Drop the original 'score' column if no longer needed
merged_df.drop(columns=['score'], inplace=True)

In [9]:
merged_df.rename(columns={
    'home_yellow_cards': 'yellow_cards_home',
    'away_yellow_cards': 'yellow_cards_away',
    'home_red_cards': 'red_cards_home',
    'away_red_cards': 'red_cards_away',
    'home_possesion': 'possesion_home',
    'away_possesion': 'possesion_away',
    'home_corners': 'corners_home',
    'away_corners': 'corners_away',
    'home_shots_on_target': 'shots_on_target_home',
    'away_shots_on_target': 'shots_on_target_away',
    'home_shots_off_target': 'shots_off_target_home',
    'away_shots_off_target': 'shots_off_target_away',
    'home_dangerous_attacks': 'dangerous_attacks_home',
    'away_dangerous_attacks': 'dangerous_attacks_away',
    'home_attacks': 'attacks_home',
    'away_attacks': 'attacks_away',
    'home_offsides': 'offsides_home',
    'away_offsides': 'offsides_away',
    'home_attempts_on_goal': 'attempts_on_goal_home',
    'away_attempts_on_goal': 'attempts_on_goal_away',
    'home_saves': 'saves_home',
    'away_saves': 'saves_away',
    'home_fauls': 'fauls_home',
    'away_fauls': 'fauls_away',
    'home_shots_blocked': 'shots_blocked_home',
    'away_shots_blocked': 'shots_blocked_away',
    'home_free_kicks': 'free_kicks_home',
    'away_free_kicks': 'free_kicks_away',
    'home_throw_ins': 'throw_ins_home',
    'away_throw_ins': 'throw_ins_away',
    'home_penalties': 'penalties_home',
    'away_penalties': 'penalties_away',
    'home_substitutions': 'substitutions_home',
    'away_substitutions': 'substitutions_away',
    'home_goal_kicks': 'goal_kicks_home',
    'away_goal_kicks': 'goal_kicks_away',
    'home_treatments': 'treatments_home',
    'away_treatments': 'treatments_away',
    'ht_score_team': 'ht_score_away'
}, inplace=True)

# Reorder columns
new_order = [
    'home_name', 'away_name', 'yellow_cards_home', 'yellow_cards_away', 'red_cards_home', 'red_cards_away',
    'possesion_home', 'possesion_away', 'corners_home', 'corners_away', 'shots_on_target_home', 'shots_on_target_away',
    'shots_off_target_home', 'shots_off_target_away', 'dangerous_attacks_home', 'dangerous_attacks_away',
    'attacks_home', 'attacks_away', 'offsides_home', 'offsides_away', 'attempts_on_goal_home', 'attempts_on_goal_away',
    'saves_home', 'saves_away', 'fauls_home', 'fauls_away', 'shots_blocked_home', 'shots_blocked_away',
    'free_kicks_home', 'free_kicks_away', 'throw_ins_home', 'throw_ins_away', 'penalties_home', 'penalties_away',
    'substitutions_home', 'substitutions_away', 'goal_kicks_home', 'goal_kicks_away', 'treatments_home', 'treatments_away', 'ht_score_home', 'ht_score_away', 'team1.overall_win_ratio', 'team1.overall_draw_ratio',
    'team1.overall_loss_ratio', 'team1.h2h_win_ratio', 'team1.h2h_draw_ratio', 'team1.h2h_loss_ratio',
    'team2.overall_draw_ratio', 'team2.overall_loss_ratio', 'team2.h2h_win_ratio', 'team2.h2h_draw_ratio', 'team2.h2h_loss_ratio'
]

merged_df = merged_df[new_order]

In [10]:
merged_df

,home_name,away_name,yellow_cards_home,yellow_cards_away,red_cards_home,red_cards_away,possesion_home,possesion_away,corners_home,corners_away,shots_on_target_home,shots_on_target_away,shots_off_target_home,shots_off_target_away,dangerous_attacks_home,dangerous_attacks_away,attacks_home,attacks_away,offsides_home,offsides_away,attempts_on_goal_home,attempts_on_goal_away,saves_home,saves_away,fauls_home,fauls_away,shots_blocked_home,shots_blocked_away,free_kicks_home,free_kicks_away,throw_ins_home,throw_ins_away,penalties_home,penalties_away,substitutions_home,substitutions_away,goal_kicks_home,goal_kicks_away,treatments_home,treatments_away,ht_score_home,ht_score_away,team1.overall_win_ratio,team1.overall_draw_ratio,team1.overall_loss_ratio,team1.h2h_win_ratio,team1.h2h_draw_ratio,team1.h2h_loss_ratio,team2.overall_draw_ratio,team2.overall_loss_ratio,team2.h2h_win_ratio,team2.h2h_draw_ratio,team2.h2h_loss_ratio
0,Floresta,Ferroviário CE,3,3,1,0,65,35,4,3,6,3,11,3,98,50,140,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0,0.166667,0.000000,0.833333,0.500000,0.000000,0.000000,0.333333,0.500000,0.000000,0.000000,0.500000
1,Bolivar,Palestino,1,2,0,0,71,28,6,1,7,6,10,9,33,25,94,71,2,5,17,15,5,6,8,6,5,3,0,0,0,0,0,0,0,0,0,0,0,0,3.0,1.0,0.500000,0.333333,0.166667,0.166667,0.000000,0.000000,0.166667,0.500000,0.000000,0.000000,0.166667
2,Flamengo,Millonarios,0,1,1,0,62,38,6,1,6,3,6,6,58,16,146,81,0,2,12,9,3,4,10,12,2,2,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0.0,0.666667,0.166667,0.166667,0.000000,0.166667,0.000000,0.666667,0.166667,0.000000,0.166667,0.000000
3,Independiente Juniors,San Antonio,4,1,0,0,39,61,4,7,5,5,5,14,39,69,110,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.333333,0.333333,0.333333,0.000000,0.166667,0.000000,0.666667,0.000000,0.000000,0.166667,0.000000
4,Coquimbo Unido,Bragantino,2,3,0,0,39,61,4,2,3,2,4,4,22,33,60,74,4,1,7,6,3,2,9,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.666667,0.166667,0.166667,0.000000,0.000000,0.166667,0.333333,0.166667,0.166667,0.000000,0.000000
5,Racing Club,Luqueno,3,3,0,0,52,48,5,2,6,0,9,2,33,11,80,69,0,0,15,2,0,4,7,8,3,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.500000,0.166667,0.333333,0.166667,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.166667
6,Real Cartagena,Atletico Huila,2,2,0,1,56,43,1,3,2,0,6,0,38,38,63,75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.333333,0.166667,0.500000,0.333333,0.166667,0.500000,0.000000,0.500000,0.500000,0.166667,0.333333
7,Delfin,Real Tomayapo,0,0,0,0,57,42,4,4,6,3,12,4,39,24,89,41,0,2,17,7,0,4,6,6,3,2,0,0,0,0,0,0,0,0,0,0,0,0,2.0,3.0,0.166667,0.166667,0.666667,0.166667,0.000000,0.000000,0.166667,0.666667,0.000000,0.000000,0.166667
8,Internacional,Belgrano Cordoba,0,3,0,0,75,25,4,0,2,4,8,1,48,16,121,44,2,1,10,4,1,1,3,14,6,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0,0.500000,0.333333,0.166667,0.000000,0.166667,0.000000,0.333333,0.166667,0.000000,0.166667,0.000000


In [11]:
import pandas as pd
# Các cột không cần chia giá trị
exclude_columns = ['home_name', 'away_name',
    'team1.overall_win_ratio', 'team1.overall_draw_ratio', 'team1.overall_loss_ratio',
    'team1.h2h_win_ratio', 'team1.h2h_draw_ratio', 'team1.h2h_loss_ratio',
    'team2.overall_draw_ratio', 'team2.overall_loss_ratio',
    'team2.h2h_win_ratio', 'team2.h2h_draw_ratio', 'team2.h2h_loss_ratio']
columns_to_divide = [col for col in merged_df.columns if col not in exclude_columns]

# Convert columns to numeric, coercing errors to NaN
merged_df[columns_to_divide] = merged_df[columns_to_divide].apply(pd.to_numeric, errors='coerce')
# Perform the division operation
merged_df[columns_to_divide] = merged_df[columns_to_divide].div(2)

In [12]:
merged_df

,home_name,away_name,yellow_cards_home,yellow_cards_away,red_cards_home,red_cards_away,possesion_home,possesion_away,corners_home,corners_away,shots_on_target_home,shots_on_target_away,shots_off_target_home,shots_off_target_away,dangerous_attacks_home,dangerous_attacks_away,attacks_home,attacks_away,offsides_home,offsides_away,attempts_on_goal_home,attempts_on_goal_away,saves_home,saves_away,fauls_home,fauls_away,shots_blocked_home,shots_blocked_away,free_kicks_home,free_kicks_away,throw_ins_home,throw_ins_away,penalties_home,penalties_away,substitutions_home,substitutions_away,goal_kicks_home,goal_kicks_away,treatments_home,treatments_away,ht_score_home,ht_score_away,team1.overall_win_ratio,team1.overall_draw_ratio,team1.overall_loss_ratio,team1.h2h_win_ratio,team1.h2h_draw_ratio,team1.h2h_loss_ratio,team2.overall_draw_ratio,team2.overall_loss_ratio,team2.h2h_win_ratio,team2.h2h_draw_ratio,team2.h2h_loss_ratio
0,Floresta,Ferroviário CE,1.5,1.5,0.5,0.0,32.5,17.5,2.0,1.5,3.0,1.5,5.5,1.5,49.0,25.0,70.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.166667,0.000000,0.833333,0.500000,0.000000,0.000000,0.333333,0.500000,0.000000,0.000000,0.500000
1,Bolivar,Palestino,0.5,1.0,0.0,0.0,35.5,14.0,3.0,0.5,3.5,3.0,5.0,4.5,16.5,12.5,47.0,35.5,1.0,2.5,8.5,7.5,2.5,3.0,4.0,3.0,2.5,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.5,0.500000,0.333333,0.166667,0.166667,0.000000,0.000000,0.166667,0.500000,0.000000,0.000000,0.166667
2,Flamengo,Millonarios,0.0,0.5,0.5,0.0,31.0,19.0,3.0,0.5,3.0,1.5,3.0,3.0,29.0,8.0,73.0,40.5,0.0,1.0,6.0,4.5,1.5,2.0,5.0,6.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.666667,0.166667,0.166667,0.000000,0.166667,0.000000,0.666667,0.166667,0.000000,0.166667,0.000000
3,Independiente Juniors,San Antonio,2.0,0.5,0.0,0.0,19.5,30.5,2.0,3.5,2.5,2.5,2.5,7.0,19.5,34.5,55.0,72.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.333333,0.333333,0.333333,0.000000,0.166667,0.000000,0.666667,0.000000,0.000000,0.166667,0.000000
4,Coquimbo Unido,Bragantino,1.0,1.5,0.0,0.0,19.5,30.5,2.0,1.0,1.5,1.0,2.0,2.0,11.0,16.5,30.0,37.0,2.0,0.5,3.5,3.0,1.5,1.0,4.5,4.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.666667,0.166667,0.166667,0.000000,0.000000,0.166667,0.333333,0.166667,0.166667,0.000000,0.000000
5,Racing Club,Luqueno,1.5,1.5,0.0,0.0,26.0,24.0,2.5,1.0,3.0,0.0,4.5,1.0,16.5,5.5,40.0,34.5,0.0,0.0,7.5,1.0,0.0,2.0,3.5,4.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.500000,0.166667,0.333333,0.166667,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.166667
6,Real Cartagena,Atletico Huila,1.0,1.0,0.0,0.5,28.0,21.5,0.5,1.5,1.0,0.0,3.0,0.0,19.0,19.0,31.5,37.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.166667,0.500000,0.333333,0.166667,0.500000,0.000000,0.500000,0.500000,0.166667,0.333333
7,Delfin,Real Tomayapo,0.0,0.0,0.0,0.0,28.5,21.0,2.0,2.0,3.0,1.5,6.0,2.0,19.5,12.0,44.5,20.5,0.0,1.0,8.5,3.5,0.0,2.0,3.0,3.0,1.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.5,0.166667,0.166667,0.666667,0.166667,0.000000,0.000000,0.166667,0.666667,0.000000,0.000000,0.166667
8,Internacional,Belgrano Cordoba,0.0,1.5,0.0,0.0,37.5,12.5,2.0,0.0,1.0,2.0,4.0,0.5,24.0,8.0,60.5,22.0,1.0,0.5,5.0,2.0,0.5,0.5,1.5,7.0,3.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.500000,0.333333,0.166667,0.000000,0.166667,0.000000,0.333333,0.166667,0.000000,0.166667,0.000000


In [13]:
import joblib
import pandas as pd
best_rf_loaded = joblib.load('/workspaces/match_football_prediction/dangdienra/best_model.joblib')

In [23]:
import numpy as np
predictions = best_rf_loaded.predict(merged_df.drop(columns=['home_name', 'away_name']))

# Map predictions to outcomes
outcome_map = {0: 'draw', 1: 'home win', -1: 'away win'}
merged_df['outcome'] = [outcome_map[pred] for pred in predictions]

# Print the results
for index, row in merged_df.iterrows():
    print(f"Home Team: {merged_df['home_name']}, Away Team: {merged_df['away_name']}, Outcome: {merged_df['outcome']}")

Home Team: 0                 Floresta
1                  Bolivar
2                 Flamengo
3    Independiente Juniors
4           Coquimbo Unido
5              Racing Club
6           Real Cartagena
7                   Delfin
8            Internacional
Name: home_name, dtype: object, Away Team: 0      Ferroviário CE
1           Palestino
2         Millonarios
3         San Antonio
4          Bragantino
5             Luqueno
6      Atletico Huila
7       Real Tomayapo
8    Belgrano Cordoba
Name: away_name, dtype: object, Outcome: 0    home win
1    home win
2        draw
3        draw
4    away win
5    home win
6    home win
7    home win
8        draw
Name: outcome, dtype: object
Home Team: 0                 Floresta
1                  Bolivar
2                 Flamengo
3    Independiente Juniors
4           Coquimbo Unido
5              Racing Club
6           Real Cartagena
7                   Delfin
8            Internacional
Name: home_name, dtype: object, Away Team: 0      Ferro

In [24]:
merged_df['outcome']

0    home win
1    home win
2        draw
3        draw
4    away win
5    home win
6    home win
7    home win
8        draw
Name: outcome, dtype: object